<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initialization" data-toc-modified-id="Initialization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialization</a></span><ul class="toc-item"><li><span><a href="#Imports-and-configurations" data-toc-modified-id="Imports-and-configurations-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports and configurations</a></span></li><li><span><a href="#Data-set-preparation:-PBMC" data-toc-modified-id="Data-set-preparation:-PBMC-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data set preparation: PBMC</a></span></li></ul></li><li><span><a href="#Run-models" data-toc-modified-id="Run-models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Run models</a></span><ul class="toc-item"><li><span><a href="#Factor-Analysis" data-toc-modified-id="Factor-Analysis-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Factor Analysis</a></span></li><li><span><a href="#ZIFA" data-toc-modified-id="ZIFA-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>ZIFA</a></span></li><li><span><a href="#pCMF" data-toc-modified-id="pCMF-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>pCMF</a></span></li><li><span><a href="#ZINB-WaVE" data-toc-modified-id="ZINB-WaVE-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>ZINB-WaVE</a></span></li><li><span><a href="#scVI" data-toc-modified-id="scVI-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>scVI</a></span></li><li><span><a href="#m-pCMF" data-toc-modified-id="m-pCMF-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>m-pCMF</a></span></li><li><span><a href="#ZINBayes" data-toc-modified-id="ZINBayes-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>ZINBayes</a></span></li></ul></li><li><span><a href="#Plot-results" data-toc-modified-id="Plot-results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Plot results</a></span><ul class="toc-item"><li><span><a href="#Latent-space-clustering" data-toc-modified-id="Latent-space-clustering-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Latent space clustering</a></span></li><li><span><a href="#Held-out-data-log-likelihood" data-toc-modified-id="Held-out-data-log-likelihood-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Held-out data log-likelihood</a></span></li><li><span><a href="#Imputation-error" data-toc-modified-id="Imputation-error-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Imputation error</a></span></li><li><span><a href="#Technical-variability" data-toc-modified-id="Technical-variability-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Technical variability</a></span></li></ul></li></ul></div>

# Initialization

## Imports and configurations

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# %matplotlib inline

from zinbayes.zinbayes import ZINBayes

from pCMF import mpCMF

from state_of_art.scVI.scvi import scVI
from state_of_art.ZIFA.zifa import ZIFA
from state_of_art.ZINB_WaVE.zinb_wave import ZINB_WaVE
from state_of_art.pCMF.pcmf import pCMF

from misc.model_wrapper import ModelWrapper
from misc import print_utils, plot_utils

import edward as ed
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from scipy.stats import gaussian_kde, pearsonr
import pandas as pd
from sklearn.decomposition import FactorAnalysis
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

from edward.models import Beta, RelaxedBernoulli, Gamma, Poisson, Dirichlet, Mixture, Categorical, TransformedDistribution, Normal, PointMass

plt.style.use('seaborn-whitegrid')

In [ ]:
# scatter plots style
s = 30
alpha = 0.5

## Data set preparation: PBMC

In [ ]:
X = pd.read_csv('../../data/PBMC/counts.csv')
y = pd.read_csv('../../data/PBMC/bio.csv')

X = X.T
X = np.array(X)
X = X[1:, :]
X = X.astype(int)

y = np.array(y)[:, 1]
y_names, y_id = np.unique(y, return_inverse=True)

var_genes = np.var(X, axis=0)
X = X[:, np.argsort(var_genes)[-1000:]]

n_cells = X.shape[0]
n_genes = X.shape[1]
n_types = np.unique(y).size

print('Data shape:')
print('{} cells, {} genes\n'.format(X.shape[0], X.shape[1]))
print('{} cell types: {}\n'.format(y_names.size, y_names))
print('Percentage of zeros in the data set:')
print('{:.2f}%'.format(np.count_nonzero(X==0) / X.size * 100))

In [ ]:
labels = y_id

In [ ]:
clusters = y

In [ ]:
library_size = np.sum(X, axis=1)

In [ ]:
detection_rate = np.sum(X != 0, axis=1)

In [ ]:
X_train, X_test, c_train, c_test = train_test_split(X, labels, test_size=0.3)

In [ ]:
N = X_train.shape[0]
N_test = X_test.shape[0]
P = n_genes
C = n_types

# Run models
For each model, we need:
* 5 repetitions on the full data set, for clustering benchmarking. 
    - Keep the best run and:
         - perform t-SNE;
         - get m-pCMF and ZINBayes' cell scalings.
* 5-fold cross-validation to compute the held-out data log-likelihood.
* Run on 5 different corrupted full data sets for imputation benchmarking.

(ensure scVI does not separate the data more than we want it to.)

In [ ]:
K = 10

## Factor Analysis

In [ ]:
obj = FactorAnalysis(n_components=K)
fa = ModelWrapper(obj, X, c_train=labels, name='FA', do_imp=False, log_data=True)
fa.run(do_silh=True, do_dll=True, do_holl=True, do_tsne=False, verbose=True)

In [ ]:
# 5 repetitions on the full data set
fa_full_reps = []

for i in range(5):
    print('Run {}...'.format(i))
    obj = FactorAnalysis(n_components=K)
    fa = ModelWrapper(obj, X, c_train=labels, name='FA', do_imp=False, log_data=True)
    fa.run(do_silh=True, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    fa_full_reps.append(fa)
    print('Done.\n')

In [ ]:
# 5-fold cross-validation 
fa_cv = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 0
for train_idx, test_idx in kf.split(X, labels):
    fold = fold + 1
    print('Fold {}...'.format(fold))
    Y_train = X[train_idx]
    c_train = labels[train_idx]
    Y_test = X[test_idx]

    obj = FactorAnalysis(n_components=K)
    fa = ModelWrapper(obj, Y_train, c_train=c_train, X_test=Y_test, name='FA', log_data=True, do_imp=False)
    fa.run(do_silh=False, do_dll=True, do_holl=True, do_tsne=False, verbose=True)
    
    fa_cv.append(fa)
    print('Done.\n')

In [ ]:
fa.train_ll, fa.test_ll

## ZIFA

In [ ]:
# 5 repetitions on the full data set
zifa_full_reps = []

for i in range(1):
    print('Run {}...'.format(i))
    obj = ZIFA(n_components=K)
    zifa = ModelWrapper(obj, X, c_train=labels, name='ZIFA', log_data=True, do_imp=False)
    zifa.run(max_iter=1000, do_silh=True, max_time=60*60, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    zifa_full_reps.append(zifa)
    print('Done.\n')

In [ ]:
# 5-fold cross-validation 
zifa_cv = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 0
for train_idx, test_idx in kf.split(X, labels):
    fold = fold + 1
    print('Fold {}...'.format(fold))
    X_train = X[train_idx]
    c_train = labels[train_idx]
    X_test = X[test_idx]

    obj = ZIFA(n_components=K)
    zifa = ModelWrapper(obj, X_train, c_train=c_train, X_test=X_test, log_data=True, name='ZIFA')
    zifa.run(max_time=60*60, max_iter=100, do_silh=False, do_dll=True, do_holl=True, do_tsne=False, verbose=True)
    
    zifa_cv.append(zifa)
    print('Done.\n')

In [ ]:
# 5 corrupted data sets
zifa_imputation = []

for i in range(1):
    print('Run {}...'.format(i))
    obj = ZIFA(n_components=K)
    zifa = ModelWrapper(obj, X, c_train=labels, X_test=None, log_data=True, name='ZIFA', do_imp=True)
    zifa.run(max_time=60*60, max_iter=100, do_silh=False, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    zifa_imputation.append(zifa)
    print('Done.\n')

## pCMF

In [ ]:
# 5 repetitions on the full data set
pcmf_full_reps = []

for i in range(1):
    print('Run {}...'.format(i))
    obj = pCMF(n_components=K)
    pcmf = ModelWrapper(obj, X, c_train=labels, name='pCMF', log_data=False, do_imp=False)
    pcmf.run(max_iter=300, do_silh=True, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    pcmf_full_reps.append(pcmf)
    print('Done.\n')

In [ ]:
# 5 repetitions on the full data set
pcmf_imputation = []

for i in range(1):
    print('Run {}...'.format(i))
    obj = pCMF(n_components=K)
    pcmf = ModelWrapper(obj, X, c_train=labels, name='pCMF', log_data=False, do_imp=True)
    pcmf.run(max_iter=300, do_silh=False, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    pcmf_imputation.append(pcmf)
    print('Done.\n')

## ZINB-WaVE

In [ ]:
# 5 repetitions on the full data set
zinbwave_full_reps = []

for i in range(1):
    print('Run {}...'.format(i))
    obj = ZINB_WaVE(n_components=K)
    zinbwave = ModelWrapper(obj, X, c_train=labels, name='ZINB-WaVE', log_data=False, do_imp=False)
    zinbwave.run(do_silh=True, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    zinbwave_full_reps.append(zinbwave)
    print('Done.\n')

In [ ]:
# 5-fold cross-validation 
zinbwave_cv = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 0
for train_idx, test_idx in kf.split(X, labels):
    fold = fold + 1
    print('Fold {}...'.format(fold))
    X_train = X[train_idx]
    c_train = labels[train_idx]
    X_test = X[test_idx]

    obj = ZINB_WaVE(n_components=K)
    zinbwave = ModelWrapper(obj, X_train, c_train=c_train, X_test=X_test, name='ZINB-WaVE', log_data=False, do_imp=False)
    zinbwave.run(do_silh=False, do_dll=True, do_holl=True, do_tsne=False, verbose=True)
    
    zinbwave_cv.append(zinbwave)
    print('Done.\n')

In [ ]:
# 5 repetitions on the full data set
zinbwave_imputation = []

for i in range(1):
    print('Run {}...'.format(i))
    obj = ZINB_WaVE(n_components=K)
    zinbwave = ModelWrapper(obj, X, c_train=labels, X_test=None, name='ZINB-WaVE', log_data=False, do_imp=True)
    zinbwave.run(max_iter=1000, do_silh=False, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    zinbwave_imputation.append(zinbwave)
    print('Done.\n')

## scVI

In [ ]:
# 5 repetitions on the full data set
scvi_full_reps = []

for i in range(5):
    print('Run {}...'.format(i))
    obj = scVI(n_components=K, n_hidden=128, n_layers=1, batch_size=128)
    scvi = ModelWrapper(obj, X, c_train=labels, name='scVI', do_imp=False)
    scvi.run(max_iter=120, max_time=60*60, do_silh=True, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    scvi_full_reps.append(scvi)
    print('Done.\n')

In [ ]:
# 5-fold cross-validation 
scvi_cv = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 0
for train_idx, test_idx in kf.split(X, labels):
    fold = fold + 1
    print('Fold {}...'.format(fold))
    X_train = X[train_idx]
    c_train = labels[train_idx]
    X_test = X[test_idx]

    obj = scVI(n_components=K, n_hidden=128, batch_size=128)
    scvi = ModelWrapper(obj, X_train, c_train=c_train, X_test=X_test, name='scVI')
    scvi.run(max_iter=120, max_time=60*60, do_silh=False, do_dll=True, do_holl=True, do_tsne=False, verbose=True)
    
    scvi_cv.append(scvi)
    print('Done.\n')

In [ ]:
# 5 corrupted data sets
scvi_imputation = []

for i in range(5):
    print('Run {}...'.format(i))
    obj = scVI(n_components=K, n_hidden=128, n_layers=1, batch_size=128)
    scvi = ModelWrapper(obj, X, c_train=labels, X_test=None, name='scVI', do_imp=True)
    scvi.run(max_iter=120, max_time=60*60, do_silh=False, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    scvi_imputation.append(scvi)
    print('Done.\n')

In [ ]:
mpcmf_full_reps[4].asw

## m-pCMF

In [ ]:
# 5 repetitions on the full data set
mpcmf_full_reps = []

for i in range(5):
    print('Run {}...'.format(i))
    mpcmf = mpCMF(X, c_train=labels, n_components=K, scalings=True, sparsity=False, 
                  zero_inflation=True, empirical_bayes=False, do_imp=False, name="m-pCMF", minibatch_size=None, 
                  nb=True)
    mpcmf.run(max_iter=1000, max_time=60*30, do_silh=True, do_holl=False, do_dll=False, tol=0.00005,
              do_tsne=False, calc_test=False, verbose=True, calc_silh=False)
    
    mpcmf_full_reps.append(mpcmf)
    print('Done.\n')

In [ ]:
# 5-fold cross-validation 
mpcmf_cv = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 0
for train_idx, test_idx in kf.split(X, labels):
    fold = fold + 1
    print('Fold {}...'.format(fold))
    X_train = X[train_idx]
    c_train = labels[train_idx]
    X_test = X[test_idx]

    mpcmf = mpCMF(X_train, Y_test=X_test, n_components=K, scalings=True, sparsity=False, 
                  zero_inflation=True, empirical_bayes=False, do_imp=False, name="m-pCMF", minibatch_size=None, 
                  nb=True)
    mpcmf.run(max_iter=1000, max_time=60*30, do_silh=False, do_holl=True, do_dll=True, tol=0.00005,
              do_tsne=False, calc_test=False, verbose=True, calc_silh=False)
    
    mpcmf_cv.append(mpcmf)
    print('Done.\n')

In [ ]:
# 5 corrupted data sets
mpcmf_imputation = []

for i in range(5):
    print('Run {}...'.format(i))
    mpcmf = mpCMF(X, n_components=K, scalings=True, sparsity=False, 
                  zero_inflation=True, empirical_bayes=False, do_imp=True, name="m-pCMF", minibatch_size=None, 
                  nb=True)
    mpcmf.run(max_iter=1000, max_time=60*30, do_silh=False, do_holl=False, do_dll=False, tol=0.00005,
              do_tsne=False, calc_test=False, verbose=True, calc_silh=False)
    
    mpcmf_imputation.append(mpcmf)
    print('Done.\n')

In [ ]:
len(mpcmf_imputation)

In [ ]:
# 5 corrupted data sets
# mpcmf_imputation = []

for i in range(2):
    print('Run {}...'.format(i))
    mpcmf = mpCMF(X, n_components=K, scalings=True, sparsity=False, 
                  zero_inflation=True, empirical_bayes=False, do_imp=True, name="m-pCMF", minibatch_size=None, 
                  nb=True)
    mpcmf.run(max_iter=1000, max_time=60*30, do_silh=False, do_holl=False, do_dll=False, tol=0.00005,
              do_tsne=False, calc_test=False, verbose=True, calc_silh=False)
    
    mpcmf_imputation.append(mpcmf)
    print('Done.\n')

## ZINBayes

In [ ]:
# 5 repetitions on the full data set
zinbayes_full_reps = []

for i in range(5):
    print('Run {}...'.format(i))
    obj = ZINBayes(n_components=K, n_mc_samples=5, test_iterations=300)
    zinbayes = ModelWrapper(obj, X, c_train=labels, name='ZINBayes', do_imp=False)
    zinbayes.run(max_iter=300, do_silh=True, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    zinbayes_full_reps.append(zinbayes)
    print('Done.\n')

In [ ]:
# 5-fold cross-validation 
zinbayes_cv = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 0
for train_idx, test_idx in kf.split(X, labels):
    fold = fold + 1
    print('Fold {}...'.format(fold))
    X_train = X[train_idx]
    c_train = labels[train_idx]
    X_test = X[test_idx]

    obj = ZINBayes(n_components=K, n_mc_samples=5, test_iterations=500)
    zinbayes = ModelWrapper(obj, X_train, c_train=c_train, X_test=X_test, name='ZINBayes', do_imp=False)
    zinbayes.run(max_iter=1000, do_silh=False, do_dll=True, do_holl=True, do_tsne=False, verbose=True)
    
    zinbayes_cv.append(zinbayes)
    print('Done.\n')

In [ ]:
# 5 repetitions on the full data set
zinbayes_imputation = []

for i in range(5):
    print('Run {}...'.format(i))
    obj = ZINBayes(n_components=K, n_mc_samples=5, test_iterations=500)
    zinbayes = ModelWrapper(obj, X, c_train=c_train, X_test=X_test, name='ZINBayes', do_imp=True)
    zinbayes.run(max_iter=300, do_silh=False, do_dll=False, do_holl=False, do_tsne=False, verbose=True)
    
    zinbayes_imputation.append(zinbayes)
    print('Done.\n')

# Plot results

In [ ]:
colors = sns.color_palette("Set1", 10, desat=.7) # this is a list of 10 colors

# We must choose what color to assign to each model
model_list_clustering = [mpcmf_full_reps, zinbayes_full_reps, zinbwave_full_reps, scvi_full_reps, pcmf_full_reps, zifa_full_reps, fa_full_reps]
# model_list_ll = [mpcmf_cv, zinbayes_cv, zinbwave_cv, scvi_cv, zifa_cv, fa_cv]
model_list_imp = [mpcmf_imputation, zinbayes_imputation, zinbwave_imputation, scvi_imputation, pcmf_imputation, zifa_imputation]

# Colors
color_list_clustering = [colors[0], colors[1], colors[2], colors[4], colors[3], colors[5], colors[6]]
hatch_list_clustering = ['//', '//', None, None, None, None, None]
color_list_ll = [colors[0], colors[1], colors[2], colors[4], colors[5], colors[6]]
hatch_list_ll = ['//', '//', None, None, None, None]
color_list_imp = [colors[0], colors[1], colors[2], colors[4], colors[3], colors[5]]
hatch_list_imp = ['//', '//', None, None, None, None, None]

In [ ]:
model_list_clustering

In [ ]:
pcmf.dropimp_err

## Latent space clustering

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = plt.axes()
ax,_,_  = plot_utils.clustering_cv(model_list_clustering, colors=color_list_clustering, show_legend=False, ylim=[0, 1],
                            title=None, ax=ax, box=False, hatches=None);
ax.figure.savefig('PBMC_clustering_bars.png', dpi=300, bbox_inches='tight')

In [ ]:
model_list_2d = [mpcmf_full_reps[0], zinbayes_full_reps[0], zinbwave_full_reps[0], scvi_full_reps[0], 
                 pcmf_full_reps[0], zifa_full_reps[0], fa_full_reps[0]]

In [ ]:
for model in model_list_2d:
    model.do_tsne()

In [ ]:
fig = plt.figure(figsize=(17, 8))
ax = plt.axes()
plot_utils.plot_2d(model_list_2d, labels, nrows=2, ncols=4, ax=ax, 
                          labels=clusters, legend=True, bbox_to_anchor=[1.2, 1.], titles=True, filename='PBMC_2d_all.png')

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = plt.axes()
plot_utils.plot_2d([model_list_2d[1]], labels, nrows=1, ncols=1, ax=ax, 
                          labels=clusters, legend=True, bbox_to_anchor=[1., 1.], titles=True, filename='PBMC_2d_ZINBayes.png')

## Held-out data log-likelihood

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.axes()
ax = plot_utils.loglikelihood_cv(model_list_ll, colors=color_list_ll, show_legend=True, hatches=None,
                            title=None, ax=ax, box=False);
ax.figure.savefig('ZEISEL_ll_legend.png', dpi=300, bbox_inches='tight')

## Imputation error

In [ ]:
plt.figure(figsize=(5, 6))
ax = plt.axes()
ax = plot_utils.imputationerr_boxplot(model_list_imp, colors=color_list_imp, 
                                 show_legend=True, ax=ax, box=False, hatches=None);
plt.ylim(0, 1.5);
ax.figure.savefig('PBMC_imp_legend.png', dpi=300, bbox_inches='tight')

## Technical variability

In [ ]:
log_library_size = np.log(np.sum(X, axis=1))

In [ ]:
from scipy.stats import pearsonr

In [ ]:
fig = plt.figure(figsize=(10,4))

ax = plt.subplot(1, 2, 1)
est_l = mpcmf_full_reps[1].est_L
plt.scatter(np.log(est_l.ravel()), log_library_size, alpha=0.5, s=30)
plt.ylabel('log library size', fontsize=14)
plt.xlabel('log cell scaling', fontsize=14)
plt.title('m-pCMF (corr={:.4f})'.format(pearsonr(np.log(est_l).ravel(), log_library_size.ravel())[0]), fontsize=14)
# plt.xlim(log_library_size.min(), log_library_size.max())

ax = plt.subplot(1, 2, 2, sharey=ax)
est_l = zinbayes_full_reps[1].model_inst.est_L
plt.scatter(np.log(est_l.ravel()), log_library_size, alpha=0.5, s=30)
# plt.ylabel('log-library size')
plt.xlabel('log cell scaling', fontsize=14)
plt.title('ZINBayes (corr={:.4f})'.format(pearsonr(np.log(est_l).ravel(), log_library_size.ravel())[0]), fontsize=14)
# plt.xlim(log_library_size.min(), log_library_size.max())

# ax = plt.subplot(1, 3, 3, sharey=ax)
# est_l = scvi_full_reps[1].model_inst.est_L
# plt.scatter(est_l.ravel(), log_library_size, alpha=0.5, s=30)
# # plt.ylabel('log-library size')
# plt.xlabel('log cell scaling', fontsize=14)
# plt.title('scVI (corr={:.4f})'.format(pearsonr(np.log(est_l).ravel(), log_library_size.ravel())[0]), fontsize=14)
# plt.xlim(log_library_size.min(), log_library_size.max())
fig.savefig('PBMC_scalings.png', dpi=300, bbox_inches='tight')
plt.show()